In [69]:
# Import Relevant librariers
import os
import pandas as pd
import numpy as np
import seaborn as sns
#Set the Working Director
os.chdir('/Users/snehamehrin/Desktop/Udacity Project')

In [70]:
#Import data
training_df=pd.read_csv('taxi_rides.csv',dtype={'distance':np.float64})
#Check the datatypes
print("DataTypes are :")
print(training_df.dtypes)
#Explore the first few rows
print("First Few rows are :")
print(training_df.head())



DataTypes are :
id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
duration                int64
distance              float64
Unnamed: 12           float64
Unnamed: 13           float64
Unnamed: 14           float64
dtype: object
First Few rows are :
          id  vendor_id pickup_datetime dropoff_datetime  passenger_count  \
0  id3487442          2   2/29/16 18:39    2/29/16 18:42                1   
1  id1145809          2   5/16/16 23:15    5/16/16 23:31                6   
2  id3499387          1   1/25/16 19:45    1/25/16 19:54                1   
3  id0753277          1   1/27/16 22:29    1/27/16 22:29                1   
4  id2134482          1   1/22/16 16:13    1/22/16 16:13                1   

   pickup_longitude  pickup_latitude

In [63]:
print("Minimum of pickup_datetime is",min(training_df['pickup_datetime']))
print("Maximum of Pickup_datetime is ",max(training_df['pickup_datetime']))
print("Minimum of dropoff_datetime is ",min(training_df['dropoff_datetime']))
print("Maximum of dropoff_datetime is",max(training_df['dropoff_datetime']))

Minimum of pickup_datetime is 1/1/16 0:00
Maximum of Pickup_datetime is  6/9/16 9:59
Minimum of dropoff_datetime is  1/1/16 0:03
Maximum of dropoff_datetime is 7/1/16 7:46


### Date Range is from Jan 2016- July 2016

In [71]:
#Remove unwanted columns
training_df.drop(columns=['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'],inplace=True)

In [65]:
#Exploring the data again
training_df.head()
#Checking for Null Values
training_df.count()

id                    1048468
vendor_id             1048468
pickup_datetime       1048468
dropoff_datetime      1048468
passenger_count       1048468
pickup_longitude      1048468
pickup_latitude       1048468
dropoff_longitude     1048468
dropoff_latitude      1048468
store_and_fwd_flag    1048468
duration              1048468
distance              1048468
dtype: int64

In [66]:
#Checking for Primary Key
training_df['id'].nunique()

1048468

### Feature Engineering

Let's calculate some key features that will help with our analysis


In [72]:
training_df['distance_to_duration'] = training_df['distance']/training_df['duration']
training_df['pickup_datetime'] = pd.to_datetime(training_df['pickup_datetime'], errors='coerce')
training_df['weekday']=training_df['pickup_datetime'].dt.dayofweek
training_df['hour']=training_df['pickup_datetime'].dt.hour
training_df['day']=training_df['pickup_datetime'].dt.day
training_df['duration_minutes']=training_df['duration']/60

training_df['price']=2.5+ training_df['distance']*1.56

### Descriptive Statistics

Descriptive statistics will help us determine the trends,outliers and distribution of the data.
We will be examining all the descriptive numerical attributes

In [73]:
#Distribution of the data
training_df[['passenger_count','duration','duration_minutes','distance','distance_to_duration','price']].describe()

,passenger_count,duration,duration_minutes,distance,distance_to_duration,price
count,1.048468e+06,1.048468e+06,1.048468e+06,1.048468e+06,1.048468e+06,1.048468e+06
mean,1.664377e+00,9.621956e+02,1.603659e+01,3.442279e+00,4.009083e-03,7.869955e+00
std,1.314244e+00,5.853297e+03,9.755495e+01,4.382064e+00,3.956651e-03,6.836020e+00
min,0.000000e+00,1.000000e+00,1.666667e-02,0.000000e+00,0.000000e+00,2.500000e+00
25%,1.000000e+00,3.970000e+02,6.616667e+00,1.232231e+00,2.535163e-03,4.422280e+00
50%,1.000000e+00,6.620000e+02,1.103333e+01,2.094639e+00,3.553663e-03,5.767637e+00
75%,2.000000e+00,1.075000e+03,1.791667e+01,3.876902e+00,4.959640e-03,8.547967e+00
max,9.000000e+00,3.526282e+06,5.877137e+04,1.240909e+03,1.910372e+00,1.938318e+03


<p>Let's look at the descriptive statistics for our numerical data</p>

### Passenger Count

* Mean quickly tells the most representative values in your data.

* 1.64 is not exactly a passenger count, but it gives you a rough idea.

* There are also couple of outliers with passenger count as 9 and passenger count as 0

* Looking at the other statistics we can see that less than 50% of Taxi rides have only one customers.

**So on an average most of the taxi riders are having passenger count as 1**

### Duration

* On an average most of the rides take 16 minutes with a standard deviation of 9.7 minutes
* 68% of the rides ranges from 7-25 minutes, which is a broad range.
* This clearly states that during some times, the duration is very long.
* Less than 50% of the rides takes almost 11 mintes.
* Minimum duration is 0.01 minutes, which is very less to be considered as a ride.

**Duration less than 1 minute**

### Distance

* Ona an average most rides have a distance of around 3 km with a standard deviation of 4
* 68% of the rides ranges from 1-7 KM.
* Less than 50% of the rides are less than 2km.
* There seems to be  a jump from 75% to 100 %- suggesting outliers.

**Distance of 0km and less than 25 KM should be removed**

In [13]:
# Taking a closer look at the outliers
training_df.loc[training_df['passenger_count']==max(training_df['passenger_count'])]

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,duration,distance,distance_to_duration,weekday,hour,day,price
1489,id2489742,2,2016-06-24 08:09:00,6/24/16 8:18,9,-73.710632,40.671581,-73.710632,40.671581,N,560,0.0,0.0,4,8,24,0.0


In [15]:
len(training_df.loc[training_df['passenger_count']==min(training_df['passenger_count'])])

39

### Cleaning Data
* Remove **Passenger count =0 and less than 5**, since most of the **passenger count is only 1-2** and I want to be able to focus only on that **sub group**. If further analysis is needed we can narrow down the group of more than 6 passengers and investigate further. But for this problem it's not needed.
* Remove very small distances **<0.1 and greater distances greater than 25 KM**. Since our target is not for longer distances such as **overnight travels**,eliminating these outliers will **reduce the noise** in the dataset.


In [51]:
#Cleaning data
training_df=training_df[(training_df['passenger_count']>=1) & (training_df['passenger_count']<=5)]
#training_df=training_df[(training_df['distance']>0.05) & (training_df['distance']<25)]
training_df=training_df[(training_df['duration']>60) & (training_df['distance']<25) & (training_df['distance']>0)]


In [52]:
#Distribution of the data
training_df[['passenger_count','duration','distance','distance_to_duration','duration_minutes']].describe()

,passenger_count,duration,distance,distance_to_duration,duration_minutes
count,1.004192e+06,1.004192e+06,1.004192e+06,1.004192e+06,1.004192e+06
mean,1.516903e+00,9.628243e+02,3.438552e+00,4.002085e-03,1.604707e+01
std,1.059396e+00,5.922067e+03,3.838729e+00,2.134311e-03,9.870111e+01
min,1.000000e+00,6.100000e+01,9.490000e-05,1.098990e-09,1.016667e+00
25%,1.000000e+00,4.010000e+02,1.250398e+00,2.550835e-03,6.683333e+00
50%,1.000000e+00,6.660000e+02,2.112351e+00,3.561455e-03,1.110000e+01
75%,2.000000e+00,1.077000e+03,3.895092e+00,4.957832e-03,1.795000e+01
max,5.000000e+00,3.526282e+06,2.499534e+01,1.621484e-01,5.877137e+04


In [74]:
#Computing Second Standard Deviation
descriptive_stats=pd.DataFrame(training_df[['passenger_count','duration','distance','distance_to_duration','duration_minutes','price']].describe())
descriptive_stats_2=descriptive_stats.mul(2,axis='std')
descriptive_stats_2.rename(index = {'std' : '2std'},inplace=True)
descriptive_stats.append((descriptive_stats_2.loc['2std']))

,passenger_count,duration,distance,distance_to_duration,duration_minutes,price
count,1.048468e+06,1.048468e+06,1.048468e+06,1.048468e+06,1.048468e+06,1.048468e+06
mean,1.664377e+00,9.621956e+02,3.442279e+00,4.009083e-03,1.603659e+01,7.869955e+00
std,1.314244e+00,5.853297e+03,4.382064e+00,3.956651e-03,9.755495e+01,6.836020e+00
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.666667e-02,2.500000e+00
25%,1.000000e+00,3.970000e+02,1.232231e+00,2.535163e-03,6.616667e+00,4.422280e+00
50%,1.000000e+00,6.620000e+02,2.094639e+00,3.553663e-03,1.103333e+01,5.767637e+00
75%,2.000000e+00,1.075000e+03,3.876902e+00,4.959640e-03,1.791667e+01,8.547967e+00
max,9.000000e+00,3.526282e+06,1.240909e+03,1.910372e+00,5.877137e+04,1.938318e+03
2std,2.628488e+00,1.170659e+04,8.764128e+00,7.913303e-03,1.951099e+02,1.367204e+01


In [52]:
#Output the dataset to tableau for easy visualization
training_df.to_csv (r'udacity_cleaned_dataset.csv', index = False, header=True)